In [58]:
class vec3f(object):
    
    def __init__(self, x,y,z):
        self.x = int(x)
        self.y = int(y)
        self.z = int(z)
        
    def step(self, v):
        self.x = self.x + v.x
        self.y = self.y + v.y
        self.z = self.z + v.z
        
    def __str__(self):
        return '<x=%i, y=%i, z=%i>' % (self.x, self.y, self.z)

class Moon(object):
    
    def __init__(self, pos_str):
        noangles = pos_str[1:-1]
        coords = {}
        for s in noangles.split(','):
            coords[s.strip().split('=')[0]] = float(s.strip()[2:])
        
        self.pos = vec3f(coords['x'], coords['y'], coords['z'])
        self.vel = vec3f(0,0,0)

    def gravitize(self, other_moon):
        if(self == other_moon):
            return
        if(other_moon.pos.x < self.pos.x):
            self.vel.x = self.vel.x - 1
        elif(other_moon.pos.x > self.pos.x):
            self.vel.x = self.vel.x + 1
        if(other_moon.pos.y < self.pos.y):
            self.vel.y = self.vel.y - 1
        elif(other_moon.pos.y > self.pos.y):
            self.vel.y = self.vel.y + 1
        if(other_moon.pos.z < self.pos.z):
            self.vel.z = self.vel.z - 1
        elif(other_moon.pos.z > self.pos.z):
            self.vel.z = self.vel.z + 1
            
    def potential(self):
        return abs(self.pos.x) + abs(self.pos.y) + abs(self.pos.z)

    def kinetic(self):
        return abs(self.vel.x) + abs(self.vel.y) + abs(self.vel.z)
    
    def total_energy(self):
        return self.potential() * self.kinetic()

In [59]:
moon = Moon('<y=-1, x=0, z=2>')
print(moon.pos.x)
print(moon.pos.y)
print(moon.pos.z)

0
-1
2


In [61]:
class System(object):
    def __init__(self, data):
        self.moons = [Moon(i) for i in data]
        
    def step(self):
        for i in range(len(self.moons)-1):
            m1 = self.moons[i]
            for j in range(i+1, len(self.moons)):
                m2 = self.moons[j]
                m1.gravitize(m2);
                m2.gravitize(m1);
        for moon in self.moons:
            moon.pos.step(moon.vel)
            
    def total_energy(self):
        energy = 0
        for moon in self.moons:
            energy = energy + moon.total_energy()
        return energy

    def __repr__(self):
        s = ''
        for moon in self.moons:
            s = s + ('pos=%s, vel=%s\n' % (str(moon.pos), str(moon.vel)))
        return s
    
    def tight_rep(self, d):
        s = ''
        
        if(d == 0):
            for moon in self.moons:
                s = s + ('%s%s' % (str(moon.pos.x), str(moon.vel.x)))
        if(d == 1):
            for moon in self.moons:
                s = s + ('%s%s' % (str(moon.pos.y), str(moon.vel.y)))
        if(d == 2):
            for moon in self.moons:
                s = s + ('%s%s' % (str(moon.pos.z), str(moon.vel.z)))
                
        return s

example1 = ['<x=-1, y=0, z=2>','<x=2, y=-10, z=-7>','<x=4, y=-8, z=8>','<x=3, y=5, z=-1>']
test_system = System(example1)
for i in range(10):
    test_system.step()
print(test_system.total_energy())

179


In [62]:
example2 = ["<x=-8, y=-10, z=0>","<x=5, y=5, z=10>","<x=2, y=-7, z=3>","<x=9, y=-8, z=-3>"]
sys2 = System(example2)
for i in range(100):
    sys2.step()
print(sys2.total_energy())

1940


In [63]:
def sim_run(data, steps):
    system = System(data)
    for i in range(steps):
        system.step()
    return system.total_energy()

print(sim_run(example2, 100))

1940


In [64]:
part1 = ["<x=5, y=4, z=4>","<x=-11, y=-11, z=-3>","<x=0, y=7, z=0>","<x=-13, y=2, z=10>"]

print('part 1: %i' % sim_run(part1, 1000))

part 1: 10845


In [77]:
def find_gcd(vals):
    smallest = vals[0]
    for val in vals[1:]:
        if(val < smallest):
            smallest = val
    gcd = 1
    for i in range(1,smallest):
        if(vals[0] % i == 0 and
           vals[1] % i == 0 and
           vals[2] % i == 0):
            gcd = i
    print("gcd = %i" % gcd)
    return gcd

def calc_primes(lessthan):
    primes = []
    for i in range(2, lessthan):
        is_prime = True
        for p in primes:
            if((i % p) == 0):
                is_prime = False
                break
        if(is_prime):
            primes.append(i)
    return primes
print(calc_primes(30))

def find_lcm(vals):
    highest = vals[0]
    for val in vals[1:]:
        if(val > highest):
            highest = val
    primes = calc_primes(highest)
    
    factorize = [{}, {}, {}]
    expos = {}
    
    for prime in primes:
        bigf = 0
        for i in range(3):
            f = 0
            r = vals[i]
            while((r % prime) == 0):
                f = f + 1
                r = r / prime
            factorize[i][prime] = f
            if(f > bigf):
                bigf = f
        if(bigf > 0):
            expos[prime] = bigf
    final = 1
    for k in expos.keys():
        final = final * (k ** expos[k])
    return final
    #return (vals[0] * vals[1] * vals[2]) / find_gcd(vals)

def sim_run_repeat(data):
    system = System(data)
    record_x = {}
    record_y = {}
    record_z = {}
    x_period = -1
    y_period = -1
    z_period = -1
    
    record_x[system.tight_rep(0)] = True
    record_y[system.tight_rep(1)] = True
    record_z[system.tight_rep(2)] = True
    
    i = 0
    while(True):
        system.step()
        i = i + 1
        if((i % 1000000) == 0):
            print(i)
        if(x_period < 0):
            tr = system.tight_rep(0)
            if(tr in record_x):
                x_period = i
            else:
                record_x[tr] = True
        if(y_period < 0):
            tr = system.tight_rep(1)
            if(tr in record_y):
                y_period = i
            else:
                record_y[tr] = True
        if(z_period < 0):
            tr = system.tight_rep(2)
            if(tr in record_z):
                z_period = i
            else:
                record_z[tr] = True
        if(x_period > -1 and y_period > -1 and z_period > -1 ):
            print('x_period=%i, y_period=%i, z_period=%i' % (x_period, y_period, z_period))
            return find_lcm([x_period, y_period, z_period])


        
print(sim_run_repeat(example1))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
x_period=18, y_period=28, z_period=44
2772


In [78]:
print(sim_run_repeat(example2))

x_period=2028, y_period=5898, z_period=4702
4686774924


In [79]:
print('part 2: %i' % sim_run_repeat(part1))

x_period=186028, y_period=231614, z_period=102356
part 2: 551272644867044
